# Geo-Safe Monitor PH: Land Use Change Analysis

This notebook runs a supervised classification over Baungon, Bukidnon, then compares land cover between 2015 and 2025.

In [1]:
import importlib.util
import subprocess
import sys

DEPENDENCIES = {
    'ee': 'earthengine-api',
    'geemap': 'geemap',
}

for module_name, package_name in DEPENDENCIES.items():
    if importlib.util.find_spec(module_name) is None:
        print(f'Installing {package_name}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])


In [2]:
import ee
import geemap

try:
    ee.Initialize()
    print('Google Earth Engine initialized.')
except Exception:
    print('Authenticating with Google Earth Engine...')
    ee.Authenticate()
    ee.Initialize()
    print('Authentication complete.')


Google Earth Engine initialized.


## Configuration and Training Data

Define ROI, land cover classes, and sample points used for the supervised classifier.

In [3]:
roi = ee.Geometry.Rectangle([124.60, 8.30, 124.70, 8.40])

GLAD_HEX = [
    'FEFECC','FAFAC3','F7F7BB','F4F4B3','F1F1AB','EDEDA2','EAEA9A','E7E792','E4E48A',
    'E0E081','DDDD79','DADA71','D7D769','D3D360','D0D058','CDCD50','CACA48','C6C63F','C3C337','C0C02F',
    'BDBD27','B9B91E','B6B616','B3B30E','B0B006','609C60','5C985C','589558','549254','508E50','4C8B4C','488848',
    '448544','408140','3C7E3C','387B38','347834','317431','2D712D','296E29','256B25','216721','1D641D','196119','155E15',
    '115A11','0D570D','095409','065106','643700','643A00','643D00','644000','644300','644600','644900','654C00','654F00',
    '655200','655500','655800','655A00','655D00','656000','656300','666600','666900','666C00','666F00','667200','667500',
    '667800','667B00','FF99FF','FC92FC','F98BF9','F685F6','F37EF3','F077F0','ED71ED','EA6AEA','E763E7','E45DE4','E156E1',
    'DE4FDE','DB49DB','D842D8','D53BD5','D235D2','CF2ECF','CC27CC','C921C9','C61AC6','C313C3','C00DC0','BD06BD','BB00BB',
    '000003','000004','000005','BFC0C0','B7BDC2','AFBBC4','A8B8C6','A0B6C9','99B3CB','91B1CD','89AFD0','82ACD2','7AAAD4',
    '73A7D6','6BA5D9','64A3DB','5CA0DD','549EE0','4D9BE2','4599E4','3E96E6','3694E9','2E92EB','278FED','1F8DF0','188AF2',
    '1088F4','0986F7','55A5A5','53A1A2','519E9F','4F9B9C','4D989A','4B9597','499294','478F91','458B8F','43888C','418589',
    '3F8286','3D7F84','3B7C81','39797E','37767B','357279','336F76','316C73','2F6970','2D666E','2B636B','296068','285D66',
    'BB93B0','B78FAC','B48CA9','B189A6','AE85A2','AA829F','A77F9C','A47B99','A17895','9E7592','9A718F','976E8C','946B88',
    '916885','8D6482','8A617F','875E7B','845A78','815775','7D5472','7A506E','774D6B','744A68','714765','DE7CBB','DA77B7',
    'D772B3','D46EAF','D169AB','CE64A8','CB60A4','C85BA0','C4579C','C15298','BE4D95','BB4991','B8448D','B54089','B23B86',
    'AF3682','AB327E','A82D7A','A52976','A22473','9F1F6F','9C1B6B','991667','961264','000000','000000','000000','1964EB',
    '1555E4','1147DD','0E39D6','0A2ACF','071CC8','030EC1','0000BA','0000BA','040464','0000FF','3051CF','000000','000000',
    '000000','000000','000000','000000','000000','000000','000000','000000','000000','000000','000000','000000','000000',
    '000000','000000','547FC4','4D77BA','466FB1','4067A7','395F9E','335895','335896','335897','FF2828','FFFFFF','D0FFFF',
    'FFE0D0','FF7D00','FAC800','C86400','FFF000','AFCD96','AFCD96','64DCDC','00FFFF','00FFFF','00FFFF','111133','000000'
]
GLAD_PALETTE = [f'#{c}' for c in GLAD_HEX]

print('Configuration set for GLAD Global Land Cover and Land Use Change analysis.')


Configuration set for GLAD Global Land Cover and Land Use Change analysis.


In [4]:
def get_glad_lcluc(year):
    assets = {
        2000: 'projects/glad/GLCLU2020/v2/LCLUC_2000',
        2005: 'projects/glad/GLCLU2020/v2/LCLUC_2005',
        2010: 'projects/glad/GLCLU2020/v2/LCLUC_2010',
        2015: 'projects/glad/GLCLU2020/v2/LCLUC_2015',
        2020: 'projects/glad/GLCLU2020/v2/LCLUC_2020',
    }
    asset_id = assets.get(year)
    if asset_id is None:
        raise ValueError(f'No GLAD land cover asset configured for {year}.')
    return ee.Image(asset_id)


In [5]:
credit_text = ('This data is made available by the Global Land Analysis and Discovery (GLAD) lab at the University of Maryland. '
               'P.V. Potapov, M.C. Hansen, A.H. Pickens, A. Hernandez-Serna, A. Tyukavina, S. Turubanova, V. Zalles, X. Li, '               'A. Khan, F. Stolle, N. Harris, X.-P. Song, A. Baggett, I. Kommareddy, A. Komareddy (2022).')
print(credit_text)
print('For more information please visit: https://glad.umd.edu/dataset/GLCLUC2020')
print('The official legend is published at: https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx')


This data is made available by the Global Land Analysis and Discovery (GLAD) lab at the University of Maryland. P.V. Potapov, M.C. Hansen, A.H. Pickens, A. Hernandez-Serna, A. Tyukavina, S. Turubanova, V. Zalles, X. Li, A. Khan, F. Stolle, N. Harris, X.-P. Song, A. Baggett, I. Kommareddy, A. Komareddy (2022).
For more information please visit: https://glad.umd.edu/dataset/GLCLUC2020
The official legend is published at: https://storage.googleapis.com/earthenginepartners-hansen/GLCLU2000-2020/v2/legend.xlsx


In [6]:
land_mask = ee.Image('projects/glad/OceanMask').lte(1)
glad_images = {year: get_glad_lcluc(year).updateMask(land_mask).clip(roi) for year in [2000, 2005, 2010, 2015, 2020]}
glad_change = ee.Image('projects/glad/GLCLU2020/v2/LCLUC').updateMask(land_mask).clip(roi)
print('GLAD Global Land Cover and Land Use images loaded successfully.')


GLAD Global Land Cover and Land Use images loaded successfully.


In [7]:
m = geemap.Map(center=[8.35, 124.65], zoom=9)
m.add_basemap('SATELLITE')
vis_params = {'min': 0, 'max': 255, 'palette': GLAD_PALETTE}
m.add_layer(glad_images[2000], vis_params, 'GLAD Land Cover 2000')
m.add_layer(glad_images[2005], vis_params, 'GLAD Land Cover 2005')
m.add_layer(glad_images[2010], vis_params, 'GLAD Land Cover 2010')
m.add_layer(glad_images[2015], vis_params, 'GLAD Land Cover 2015')
m.add_layer(glad_images[2020], vis_params, 'GLAD Land Cover 2020')
m.add_layer(glad_change, vis_params, 'GLAD Land Cover Change 2000-2020')
legend_items = {
    'Forest → Cropland': '#E67E22',
    'Forest → Bare Ground': '#C0392B',
    'Forest → Grassland': '#27AE60'
}
m.add_legend(title='Forest Conversion Drivers (2000→2020)', legend_dict=legend_items)
m.addLayerControl()
print('Displaying GLAD Global Land Cover and Land Use Change map ...')
m


Displaying GLAD Global Land Cover and Land Use Change map ...


Map(center=[8.35, 124.65], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…